# <center> Data <center>
Este notebook explica cómo utilizamos el dataframe creado por **Adapter** para resolver la cinemática.

**Data** se encarga de:
- Eliminar el ruido.
- Calcular velocidad y aceleración a partir de posición.
- Obtener coordenadas polares.
- Calcular ángulos que nos pueden resultar útiles más adelante.


#### Inicialización

In [2]:
import pandas as pd
import numpy as np
import filter as ft
import column_filter as cf
import pickle as pkl
from video import Video
from adapter import Adapter
import plotly.express as px

video_name = "video2"
video = Video(f"videos/{video_name}.mp4")
with open(f"keypoints/{video_name}", "rb") as f:
    positions = pkl.load(f)
objects = ["shoulder", "elbow", "wrist"]
height = video.get_height()
adapter = Adapter(positions, objects, video.get_height())
df = adapter.get_adapted_data()
df

,rx_shoulder,ry_shoulder,rx_elbow,ry_elbow,rx_wrist,ry_wrist
0,150,582.0,119,410.0,123,277.0
1,151,582.0,120,410.0,124,277.0
2,151,583.0,120,411.0,124,279.0
3,151,583.0,120,411.0,125,280.0
4,150,583.0,120,411.0,126,279.0
...,...,...,...,...,...,...
844,124,575.0,115,415.0,191,457.0
845,125,575.0,115,414.0,191,455.0
846,125,575.0,115,414.0,192,450.0
847,125,575.0,115,413.0,193,448.0


#### Eliminación del ruido en la posición
Actualmente las posiciones se ven así:

In [8]:
fig = px.line(df, x = df.index, y = ['rx_wrist', 'ry_wrist'])
fig.show()

Para eliminar los picos las filtramos

In [9]:
df = ft.apply_filter(df, df.columns)
fig = px.line(df, x = df.index, y = ['rx_wrist', 'ry_wrist'])
fig.show()

#### Cálculo de la velocidad
Ahora que filtramos las posiciones podemos utilizarlas para calcular la velocidad

In [10]:
for object in objects:
    df[f"vx_{object}"] = df[f"rx_{object}"].diff()
    df[f"vy_{object}"] = df[f"ry_{object}"].diff()
fig = px.line(df, x = df.index, y = ['vx_wrist', 'vy_wrist'])
fig.show()

Como nos quedan picos, filtramos también la velocidad

In [11]:
df = ft.apply_filter(df, cf.velocity_columns(df.columns))
fig = px.line(df, x = df.index, y = ['vx_wrist', 'vy_wrist'])
fig.show()

#### Cálculo de la aceleración
Ahora podemos utilizar la velocidad para obtener la aceleración.

In [13]:
for object in objects:
    df[f"ax_{object}"] = df[f"vx_{object}"].diff()
    df[f"ay_{object}"] = df[f"vy_{object}"].diff()
fig = px.line(df, x = df.index, y = ['ax_wrist', 'ay_wrist'])
fig.show()

También debemos filtrarla.

In [14]:
df = ft.apply_filter(df, cf.acceleration_columns(df.columns))
fig = px.line(df, x = df.index, y = ['ax_wrist', 'ay_wrist'])
fig.show()

#### Obtención de coordenadas polares
Utilizamos como origen la posición del codo y el eje pasa por el hombro.

Primero calculamos los vectores desde al code al hombro y a la muñeca utilizando $\overrightarrow{AB}=(B_x-A_x)\hat{x} + (B_y-A_y)\hat{y}$

In [15]:
x_vector_elbow_to_wrist = df['rx_wrist'] - df['rx_elbow']
y_vector_elbow_to_wrist = df['ry_wrist'] - df['ry_elbow']
x_vector_elbow_to_shoulder = df['rx_shoulder'] - df['rx_elbow']
y_vector_elbow_to_shoulder = df['ry_shoulder'] - df['ry_elbow']

Luego calculamos sus módulos usando pitágoras $|\vec{A}|^2={A_x}^2+{A_y}^2$

In [16]:
magnitude_vector_elbow_to_wrist = np.sqrt(x_vector_elbow_to_wrist ** 2 + y_vector_elbow_to_wrist ** 2)
magnitude_vector_elbow_to_shoulder = np.sqrt(x_vector_elbow_to_shoulder ** 2 + y_vector_elbow_to_shoulder ** 2)

Con esto ya tenemos el $r$ de la posición de la muñeca en polares

In [17]:
df['r_wrist'] = magnitude_vector_elbow_to_wrist
fig = px.line(df, x = df.index, y = 'r_wrist')
fig.show()

Para calcular el $\theta$ usamos el producto escalar $A_xB_x+A_yB_y=|\vec{A}||\vec{B}|\cos\theta$

In [19]:
elbow_to_wrist_scalar_product_elbow_to_shoulder = x_vector_elbow_to_wrist * x_vector_elbow_to_shoulder + y_vector_elbow_to_wrist * y_vector_elbow_to_shoulder
cosine_angle = elbow_to_wrist_scalar_product_elbow_to_shoulder / (magnitude_vector_elbow_to_wrist * magnitude_vector_elbow_to_shoulder)
df['theta_wrist'] = np.arccos(cosine_angle)
fig = px.line(df, x = df.index, y = 'theta_wrist')
fig.show()

También filtramos estas funciones.

In [20]:
df = ft.apply_filter(df, ['r_wrist'])
df = ft.apply_filter(df, ['theta_wrist'])
fig = px.line(df, x = df.index, y = ['r_wrist', 'theta_wrist'])
fig.show()

Derivando $\theta$ podemos obtener la velocidad angular

In [21]:
df['angular_velocity'] = df['theta_wrist'].diff()
fig = px.line(df, x = df.index, y = 'angular_velocity')
fig.show()

In [22]:
df = ft.apply_filter(df, ['angular_velocity'])
fig = px.line(df, x = df.index, y = 'angular_velocity')
fig.show()

Y a partir de esta obtenemos la aceleración angular.

In [23]:
df['angular_acceleration'] = df['angular_velocity'].diff()
fig = px.line(df, x = df.index, y = 'angular_acceleration')
fig.show()

In [24]:
df = ft.apply_filter(df, ['angular_acceleration'])
fig = px.line(df, x = df.index, y = 'angular_acceleration')
fig.show()

#### Cálculo de ángulos que podemos necesitar luego
Obtenemos el ángulo entre el brazo y una vértical, y el ángulo entre el antebrazo y una horizontal utilizando $\sin\alpha=\frac{\text{opuesto}}{\text{hipotenusa}}$

In [25]:
distance_shoulder_vertical = df['rx_shoulder'] - df['rx_elbow']
sin_angle_upperarm_vertical = distance_shoulder_vertical / magnitude_vector_elbow_to_shoulder
df['angle_upperarm_vertical'] = np.arcsin(sin_angle_upperarm_vertical)

distance_wrist_horizontal = df['ry_wrist'] - df['ry_elbow']
sin_angle_forarm_horizontal = distance_wrist_horizontal / magnitude_vector_elbow_to_wrist
df['angle_forearm_horizontal'] = np.arcsin(sin_angle_forarm_horizontal)

fig = px.line(df, x = df.index, y = ['angle_upperarm_vertical', 'angle_forearm_horizontal'])
fig.show()

Utilizando estos ángulos y $\theta$ obtenemos los siguientes:

In [26]:
df['angle_forearm_vertical'] = df['theta_wrist'] + df['angle_upperarm_vertical']
df['angle_upperarm_horizontal'] = df['theta_wrist'] + df['angle_forearm_horizontal']
fig = px.line(df, x = df.index, y = ['angle_forearm_vertical', 'angle_upperarm_horizontal'])
fig.show()

Además de los ángulos calculamos la posición del punto medio del antebrazo, donde asumimos que está su centro de masa.

In [27]:
df['rx_forearm'] = df['rx_elbow'] + ((df['rx_wrist'] - df['rx_elbow']) / 2)
df['ry_forearm'] = df['ry_elbow'] + ((df['ry_wrist'] - df['ry_elbow']) / 2)

fig = px.line(df, x = df.index, y = ['rx_forearm', 'ry_forearm'])
fig.show()